# Multiclass Classification with a neural network

In [10]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), ".."))

In [11]:
from pyldl.activations import Sigmoid, Softmax, Tanh
from pyldl.layers import Sequential
from pyldl.losses import CrossEntropyLoss
from pyldl.modules import Linear
from pyldl.optimizers import SGD
from pyldl.utils import encode_one_hot

## Dataset and Network

In [12]:
from sklearn.datasets import load_digits

digits = load_digits()

X = digits.data
y = digits.target
y_one_hot = encode_one_hot(y)

In [13]:
network = Sequential(Linear(X.shape[1], 128), Sigmoid(), Linear(128, 64), Sigmoid(), Linear(64, 10), Softmax())
loss = CrossEntropyLoss()

## Model performances

In [14]:
SGD(network, loss, X, y_one_hot, batch_size=64, gradient_step=1e-3, n_iter=1000)

ValueError: operands could not be broadcast together with shapes (64,) (64,10) 

In [ ]:
def predict(X):
    return network.forward(X).argmax(axis=1)

def accuracy(y, yhat):
    return (y == yhat).mean()

yhat = predict(X)
print(f"Accuracy: {accuracy(y, yhat)}")